# Kaggle Categorical competition

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import metrics
import xgboost
import lightgbm
import catboost
from catboost import CatBoostClassifier
from warnings import filterwarnings
import pickle
filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')
df_sample=pd.read_csv('sample_submission.csv')

In [3]:
df_sample.head()

,id,target
0,600000,0.5
1,600001,0.5
2,600002,0.5
3,600003,0.5
4,600004,0.5


In [4]:
df_train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [5]:
df_train.isna().any()

id        False
bin_0      True
bin_1      True
bin_2      True
bin_3      True
bin_4      True
nom_0      True
nom_1      True
nom_2      True
nom_3      True
nom_4      True
nom_5      True
nom_6      True
nom_7      True
nom_8      True
nom_9      True
ord_0      True
ord_1      True
ord_2      True
ord_3      True
ord_4      True
ord_5      True
day        True
month      True
target    False
dtype: bool

In [6]:
df_train.isna().sum()

id            0
bin_0     17894
bin_1     18003
bin_2     17930
bin_3     18014
bin_4     18047
nom_0     18252
nom_1     18156
nom_2     18035
nom_3     18121
nom_4     18035
nom_5     17778
nom_6     18131
nom_7     18003
nom_8     17755
nom_9     18073
ord_0     18288
ord_1     18041
ord_2     18075
ord_3     17916
ord_4     17930
ord_5     17713
day       17952
month     17988
target        0
dtype: int64

In [7]:
df_train['bin_0'].unique()

array([ 0.,  1., nan])

In [8]:
df_train['nom_1'].unique()

array(['Trapezoid', 'Star', nan, 'Circle', 'Triangle', 'Polygon',
       'Square'], dtype=object)

In [9]:
for col in df_train.columns:
    print('column {} ----- {} unique values ---- type: {} '.format(col, df_train[col].nunique(), df_train[col].dtype.name))
#     df_train[col].unique()

column id ----- 600000 unique values ---- type: int64 
column bin_0 ----- 2 unique values ---- type: float64 
column bin_1 ----- 2 unique values ---- type: float64 
column bin_2 ----- 2 unique values ---- type: float64 
column bin_3 ----- 2 unique values ---- type: object 
column bin_4 ----- 2 unique values ---- type: object 
column nom_0 ----- 3 unique values ---- type: object 
column nom_1 ----- 6 unique values ---- type: object 
column nom_2 ----- 6 unique values ---- type: object 
column nom_3 ----- 6 unique values ---- type: object 
column nom_4 ----- 4 unique values ---- type: object 
column nom_5 ----- 1220 unique values ---- type: object 
column nom_6 ----- 1519 unique values ---- type: object 
column nom_7 ----- 222 unique values ---- type: object 
column nom_8 ----- 222 unique values ---- type: object 
column nom_9 ----- 2218 unique values ---- type: object 
column ord_0 ----- 3 unique values ---- type: float64 
column ord_1 ----- 5 unique values ---- type: object 
column ord

### Applying labelencoding to all columns

In [10]:
# le = LabelEncoder()
# df_train['bin_0'] = le.fit_transform(df_train['bin_0'])
# df_train['bin_0'].head()

In [11]:
# df_train['bin_0'].nunique()

In [12]:
# df_train['bin_3'].unique()

In [13]:
# df_train['bin_3'].head()

### A lable encoder needs to keep null values while encoding everything else. This **[link](https://stackoverflow.com/questions/54444260/labelencoder-that-keeps-missing-values-as-nan)** explains how it can be implemented.

In [14]:
encoders = dict()

for col_name in df_train.columns:
    series = df_train[col_name]
    label_encoder = LabelEncoder()
    df_train[col_name] = pd.Series(
        label_encoder.fit_transform(series[series.notnull()]),
        index=series[series.notnull()].index
    )
    encoders[col_name] = label_encoder

In [15]:
df_train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,5.0,...,27.0,2.0,0.0,3.0,2.0,20.0,56.0,5.0,2.0,0
1,1,1.0,1.0,0.0,0.0,1.0,2.0,3.0,0.0,NaN,...,2112.0,2.0,2.0,5.0,4.0,23.0,150.0,6.0,6.0,0
2,2,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,0.0,...,NaN,2.0,NaN,2.0,13.0,15.0,105.0,4.0,8.0,0
3,3,NaN,0.0,0.0,0.0,0.0,2.0,0.0,3.0,3.0,...,2167.0,0.0,4.0,4.0,0.0,2.0,NaN,2.0,2.0,0
4,4,0.0,NaN,0.0,1.0,0.0,2.0,5.0,3.0,2.0,...,1747.0,2.0,2.0,1.0,7.0,2.0,50.0,4.0,11.0,0


### Applying an xgboost to the dataframe

In [74]:
X = df_train.drop(['id', 'target'], axis=1)
y = df_train['target']

In [75]:
test_size = 0.3
train_size = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=2021)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

xgb = xgboost.XGBRegressor(n_estimator=20000, max_depth=8, colsample_bytree=0.85,
                           reg_alpha=0.05, reg_lambda=0.05, random_state=2021,learning_rate=0.008, verbosity=1)

In [76]:
xgb.fit(X_train, y_train, eval_set=[(X_val,y_val)], early_stopping_rounds=50, verbose=50 )

[00:45:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.49823
[50]	validation_0-rmse:0.43370
[99]	validation_0-rmse:0.40082


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.85, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.008, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimator=20000, n_estimators=100, n_jobs=8, num_parallel_tree=1,
             random_state=2021, reg_alpha=0.05, reg_lambda=0.05,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=1)

#### Accuracy is not the right metric for this model. 

[Why accuracy is not the right metric for regression models is explained here.](https://stackoverflow.com/questions/38015181/accuracy-score-valueerror-cant-handle-mix-of-binary-and-continuous-target/54458777) 

trying to use a classification metric (accuracy) in a regression (i.e. numeric prediction) model (LinearRegression), which is meaningless

In [77]:
xgb.score(X_test, y_test)

-0.053966823227433514

In [78]:
xgb.predict(X_test)

array([0.23654704, 0.3634557 , 0.3075731 , ..., 0.31198728, 0.31884414,
       0.32598814], dtype=float32)

In [79]:
y_test.unique()

array([0, 1], dtype=int64)

In [80]:
metrics.accuracy_score(y_test, xgb.predict(X_test).round())

0.8134777777777777

### Model score

A set of right type of metrics for regression is available [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [81]:
pred = xgb.predict(X_test)
metrics.r2_score(y_test, pred)

-0.053966823227433514

In [82]:
metrics.explained_variance_score(y_test, pred)

0.07256758979575295

In [83]:
metrics.max_error(y_test, pred)

0.7621162235736847

In [84]:
metrics.mean_absolute_error(y_test, pred)

0.378106585939063

In [85]:
metrics.mean_squared_error(y_test, pred)

0.16096742138774808

In [86]:
metrics.mean_poisson_deviance(y_test, pred)

0.6624556976566999

In [87]:
# metrics.mean_gamma_deviance(y_test, pred)
# ValueError: Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.

In [88]:
metrics.mean_tweedie_deviance(y_test, pred)

0.16096742138774808

In [73]:
metrics.mean_poisson_deviance(y_test, pred)

0.6622099630110054

## Changing encoding

nom_5: count encoding
nom_6: count encoding
nom_7: count encoding
nom_8: count encoding
nom_9: count encoding


In [55]:
df_train_2 = pd.read_csv('Train.csv')

In [60]:
df_train_2['nom_7']

0         598080a91
1         1dddb8473
2         5ddc9a726
3         3a33ef960
4         bc9cc2a94
            ...    
599995    a7059911d
599996    83bdea3a5
599997    114b1dbf3
599998    fba315672
599999    67a8d4ebb
Name: nom_7, Length: 600000, dtype: object

In [91]:
# !pip install category_encoders

In [90]:
import category_encoders as ce

In [94]:
cat_features = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
for col in cat_features:
    count_enc = ce.CountEncoder()
    df_train_2[col] = count_enc.fit_transform(df_train_2[col])

In [95]:
df_train_2.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,207,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,530,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,18073,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,149,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,560,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0
